# 📊 PyFlow: Deep PYUSD Analysis with Google Cloud's Premium RPC Methods

**Hackathon Context:** This notebook is developed for the **PayPal x Google Cloud Web3 Bounty**, demonstrating how **Google Cloud Platform's Blockchain Node Engine** unlocks powerful, cost-effective analysis of the **PayPal USD (PYUSD)** stablecoin on Ethereum.

---

## The Challenge: Unlocking Deep Blockchain Insights

Understanding the intricate movements, internal logic, and real-world interactions of stablecoins like **PYUSD** often requires deep, computationally intensive blockchain analysis. Standard block explorers and basic RPC calls provide only surface-level data, while accessing advanced tracing and state inspection methods on most platforms is prohibitively expensive or heavily rate-limited due to their high computational demands.

## The Solution: PyFlow leveraging GCP's Advantage

This notebook, **PyFlow**, provides a comprehensive toolkit for advanced PYUSD analysis by specifically utilizing **Google Cloud Platform's premium RPC debug and trace methods**.

> **🚀 GCP's Unique Offering: Cost-Effective Access to High-Multiplier Methods**
>
> Many advanced RPC methods carry significant **request multipliers** due to their computational intensity. For example, a method with a `50x` multiplier consumes the equivalent quota/cost of 50 basic calls (like `eth_call`). Methods like `trace_replayTransaction` have an even higher `100x` multiplier.
>
> **GCP's Blockchain Node Engine stands out by offering generous free quotas even for these high-multiplier methods**, effectively democratizing access to capabilities previously reserved for specialized infrastructure or high budgets.

This allows PyFlow to perform analysis typically infeasible elsewhere, such as:

*   **Forensic Accounting:** Tracing PYUSD flow through complex multi-contract DeFi interactions using methods like `debug_traceTransaction` (`50x`).
*   **Gas Optimization Analysis:** Pinpointing exact gas costs within internal PYUSD functions or integrations.
*   **Security Investigations:** Replaying failed transactions (`trace_replayTransaction`, `100x`) or examining state changes (`stateDiff` via replay).
*   **Smart Contract Auditing:** Verifying internal logic, storage layout (`debug_storageRangeAt`, `50x`), and event emission (`eth_getLogs`, `50x`).
*   **Network Health Insights:** Analyzing pending transaction queues (`txpool_status`, `50x`) and estimating confirmation times.

## 🛠️ Methods Explored (with GCP Request Multipliers):

This notebook provides practical implementations and analysis using the following GCP-powered methods for PYUSD on Ethereum. Multipliers indicate the relative request cost compared to a standard call:

*   **Detailed Tracing:**
    *   `debug_traceTransaction` (`50x`): In-depth EVM execution trace (using `callTracer` & `structLog`).
    *   `trace_transaction` (`50x`): Alternative transaction tracing method.
*   **Block-Level Analysis:**
    *   `trace_block` (`50x`, Mainnet only): Trace all transactions within a specified block.
    *   `debug_traceBlockByNumber` / `debug_traceBlockByHash` (`50x`): Alternative block tracing.
*   **State Replay & Simulation:**
    *   `trace_replayTransaction` (`100x`, Mainnet only): Re-execute a past transaction with tracers.
    *   `trace_replayBlockTransactions` (`100x`, Mainnet only): Re-execute all transactions in a block with tracers.
    *   `trace_call` (`50x`, Mainnet only): Simulate transaction calls without sending to the network.
*   **State & Data Retrieval:**
    *   `eth_getLogs` (`50x`): Efficiently fetch specific PYUSD events (e.g., Transfers, Approvals).
    *   `eth_getCode` (`10x`): Retrieve deployed contract bytecode.
    *   `debug_storageRangeAt` (`50x`): Inspect raw contract storage slots.
    *   `eth_getProof` (`50x`): Fetch Merkle proofs for state verification.
*   **Network Monitoring:**
    *   `txpool_status` (`50x`): Analyze pending/queued transaction counts.

*(Note: Multipliers are based on GCP documentation and highlight the computational intensity absorbed by the service.)*

---

**💡 Goal:** By the end of this notebook, you will understand how to leverage GCP's unique RPC capabilities, including high-multiplier methods offered with generous quotas, to perform advanced, cost-effective blockchain intelligence specifically tailored for the PYUSD stablecoin.


## 🛠️ Environment Setup: Installing Dependencies for PyFlow
---

This cell installs the necessary Python packages to run the PyFlow analysis notebook. It sets up a complete environment for interacting with the Ethereum blockchain (via GCP), analyzing PYUSD data, generating visualizations, and connecting to Google Cloud services.

### 📊 Key Dependencies & Purpose:

| Category                 | Packages                                                       | Purpose                                                             |
| :----------------------- | :------------------------------------------------------------- | :------------------------------------------------------------------ |
| **Core Blockchain/Data** | `web3`, `pandas`, `numpy`                                    | Ethereum RPC interaction, data manipulation                         |
| **Visualization**        | `matplotlib`, `plotly`, `seaborn`, `networkx`, `graphviz`    | Charts, transaction graphs, visual analysis                         |
| **Google Cloud**         |  `gspread`, `oauth2client` | Accessing Google Sheets export, Authentication |
| **Ethereum Utilities**   | `eth-utils`, `rlp`, `tqdm`                                   | Cryptographic functions, RLP encoding, progress bars                |
| **Notebook Enhancement** | `ipywidgets`, `rich`                                         | Interactive controls, improved console output                     |

### ⚙️ Runtime Notes:

*   **Environment:** Designed primarily for Google Colab.
*   **Resources:** A standard Colab runtime is usually sufficient, but a High-RAM runtime is recommended for analyzing very large blocks or complex transaction traces. GPU is generally not required.
*   **Colab Features:** The setup automatically installs system-level `graphviz` and enables interactive data tables within Colab.

> **⏳ Installation Time:** The process uses `pip` and typically completes in **1-2 minutes**. Please ensure this cell executes successfully before proceeding.

In [ ]:
# =============================================================================================
# 🛠️ Environment Setup and Package Installation
# =============================================================================================
# This cell installs and configures all necessary packages for blockchain data analysis.
# The setup process may take 1-2 minutes to complete.

import sys
import subprocess
import time
from IPython.display import clear_output
from rich.console import Console
from rich.theme import Theme
from rich.progress import Progress, SpinnerColumn, TextColumn

# Auto-adapting color theme that works well in both light and dark terminals
custom_theme = Theme({
    "info": "cyan3",          # Informational messages
    "success": "spring_green3", # Success indicators
    "warning": "gold3",       # Warning messages
    "error": "red3",          # Error messages
    "highlight": "royal_blue1"  # Highlighted information
})

# Create console with auto color system detection for better visual feedback
console = Console(theme=custom_theme)

console.print("\n✨ Environment Setup and Package Installation ✨", style="bold cyan3")
console.print("─────────────────────────────────────────────────", style="cyan3")

console.print("🔄 Starting package installation process...", style="info")

# Function to install packages and handle errors with better formatting
# This provides visual feedback during the installation process
def install_packages(packages, description):
    """Install specified packages with progress indicator and error handling"""
    with Progress(
        SpinnerColumn(),
        TextColumn(f"[info]Installing {description}..."),
        transient=True,
    ) as progress:
        task = progress.add_task("", total=None)
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages.split())
            console.print(f"✓ {description} installed", style="success")
            return True
        except subprocess.CalledProcessError:
            console.print(f"❌ Error installing {description}", style="error")
            return False

# =============================================================================================
# Core Libraries Installation
# =============================================================================================

# Install core data processing libraries
# - web3: For blockchain interaction and smart contract calls
# - pandas: For data manipulation and analysis
# - numpy: For numerical operations
# - matplotlib: For basic visualization
success = install_packages("web3==6.11.1 pandas numpy matplotlib",
                         "Core data libraries (web3, pandas, numpy, matplotlib)")

# Install advanced visualization and analysis libraries
# - plotly: For interactive charts
# - seaborn: For statistical visualizations
# - networkx: For blockchain transaction network analysis
if success:
    success = install_packages("plotly seaborn networkx",
                             "Visualization and analysis libraries (plotly, seaborn, networkx)")

# Install visualization export libraries
# - kaleido: For high-quality Plotly chart exports to PNG/PDF/SVG
if success:
    success = install_packages("kaleido",
                             "Visualization export library (required for exporting charts to images)")

# Install Google API libraries for data access and storage
# - gspread: For Google Sheets integration
# - oauth2client: For authentication with Google services
if success:
    success = install_packages("gspread oauth2client",
                             "Google API libraries (gspread, oauth2client)")

# Install interactive widgets for Jupyter/Colab notebooks
# - ipywidgets: For creating interactive controls and dashboards
if success:
    success = install_packages("ipywidgets",
                             "Interactive widgets for Jupyter notebooks")

# Install Ethereum proof verification and analysis libraries
# - eth-utils: For cryptographic functions and general Ethereum utilities
# - rlp: For Recursive Length Prefix encoding used in Ethereum
# - tqdm: For progress visualization in notebook environments
# - graphviz: For visualizing Merkle proofs and contract structures
if success:
    success = install_packages("eth-utils rlp tqdm graphviz",
                             "Ethereum proof verification libraries")

# For Colab environments, install system-level graphviz for visualization

try:
    # Check if running in Google Colab
    import google.colab
    console.print("\n\n🔄 Installing system dependencies for visualization...", style="info")
    # Install graphviz system package (used for rendering graphs)
    subprocess.check_call(['apt-get', '-qq', 'install', 'graphviz'])
    console.print("✓ System-level graphviz installed for advanced visualizations", style="success")

    # Enable enhanced data visualization and export capabilities
    console.print("\n\n🔄 Enabling enhanced data visualization and export...", style="info")

    # Enable interactive data tables from Google Colab
    try:
        from google.colab import data_table
        data_table.enable_dataframe_formatter()
        console.print("✓ Interactive data tables enabled", style="success")
    except ImportError:
        console.print("⚠️ Could not enable interactive data tables", style="warning")

    # Import additional components for file exports and Google Sheets integration
    try:
        # For direct CSV/JSON downloads
        import base64
        import io

        # For Google Sheets export
        from google.colab import output
        from googleapiclient.discovery import build
        from googleapiclient.http import MediaInMemoryUpload

        console.print("✓ Export functionality enabled", style="success")
    except ImportError:
        console.print("⚠️ Some export functions may be limited", style="warning")

    # Verify data table display
    try:
        from IPython.display import display, HTML
        console.print("✓ Data table display verified", style="success")
    except:
        console.print("⚠️ Data table display verification failed", style="warning")

except ImportError:
    console.print("ℹ️ Not running in Colab, skipping system-level installations", style="info")
except Exception as e:
    console.print(f"⚠️ Note: Could not install graphviz system package: {e}. Some visualizations may be limited.", style="warning")

# Final status message with extra spacing
if success:
    console.print("\n\n📦 ✓ All required packages installed successfully!", style="success")
else:
    console.print("\n\n⚠️ [bold]Some packages failed to install. Please check the errors above.[/bold]", style="warning")

# =============================================================================================
# Environment Verification
# =============================================================================================
# Verify all packages imported correctly and set up the analytics environment

try:
    # Core data and utility libraries
    import os
    import json
    import time
    import warnings
    import hashlib

    # Data analysis stack
    import numpy as np
    import pandas as pd

    # Network and graph analysis
    import networkx as nx

    # Visualization libraries
    import matplotlib.pyplot as plt
    import seaborn as sns
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # Date handling
    from datetime import datetime, timedelta

    # Collections and data structures
    from collections import defaultdict, Counter

    # Blockchain interaction libraries
    from web3 import Web3
    from web3.exceptions import TransactionNotFound
    from web3.middleware import geth_poa_middleware
    from hexbytes import HexBytes

    # Google Cloud and authentication (for gspread)
    from google.colab import auth # Kept for potential gspread auth if needed in Colab
    from google.oauth2 import service_account
    import gspread
    from oauth2client.client import GoogleCredentials
    from oauth2client.service_account import ServiceAccountCredentials

    # Import ipywidgets components for interactive dashboards
    import ipywidgets as widgets
    from ipywidgets import interact, interactive, fixed, interact_manual

    # Rich components for improved CLI-style output
    from rich.panel import Panel
    from rich.syntax import Syntax
    from rich.table import Table

    # Ethereum specific utilities for cryptography and data structures
    from eth_utils import keccak, to_bytes, to_hex
    import rlp
    from tqdm.notebook import tqdm
    from graphviz import Digraph

    # Suppress warnings for cleaner output
    warnings.filterwarnings('ignore')

    # Configure pandas display settings for better readability
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.max_rows', 100)      # Reasonable number of rows
    pd.set_option('display.float_format', '{:.6f}'.format)  # Format for amounts

    # Setup Plotly for better Jupyter/Colab integration
    import plotly.io as pio
    pio.templates.default = "plotly_white"

    console.print("\n\n🚀 ✓ Setup complete! Analytics platform initialized.", style="success")
except ImportError as e:
    console.print(f"\n\n❌ Error importing libraries: {e}", style="error")
    console.print("⚠️ Some required packages may not have been installed correctly.", style="warning")

## 🔑 Configuration & Authentication: Connecting to GCP and Ethereum RPC
---

This crucial cell configures PyFlow to connect to Google Cloud Platform services (for authentication and Google Sheets) and the necessary Ethereum networks via GCP's Blockchain Node Engine. **You MUST edit this cell with your specific credentials before running it.**

### 📋 Step 1: Provide Your GCP Credentials

To use GCP's Blockchain RPC and potentially Google Sheets, you need:

1.  **Your GCP Project ID:**
    *   **Purpose:** Identifies your Google Cloud project, primarily used here to scope authentication requests for Google Drive/Sheets access.
    *   **How to Obtain:** If you don't have one, create it at [GCP Console](https://console.cloud.google.com/projectcreate).
    *   **➡️ ACTION REQUIRED:** Find the `GCP_PROJECT_ID` variable in the code below and replace `"YOUR_PROJECT_ID"` with your actual Project ID string.
2.  **Your GCP Blockchain RPC Endpoints (with API Key):**
    *   **Purpose:** Secure URLs to connect to Ethereum Mainnet and testnets via GCP. This is key to interacting with the blockchain using `web3.py`.
    *   **How to Obtain:**
        1.  Go to the [GCP Blockchain Node Engine Console](https://console.cloud.google.com/blockchain/node-engine) in your project.
        2.  Enable the API if you haven't already.
        3.  Copy the **full HTTPS RPC endpoint URL** (including `?key=...`) for **Ethereum Mainnet**. Optionally, copy URLs for testnets (Holesky, Sepolia) if needed.
    *   **➡️ ACTION REQUIRED:** Find the `BLOCKCHAIN_RPC` dictionary in the code below. Replace the placeholder URLs (e.g., `"https://blockchain.googleapis.com/..."`) with your *complete* copied endpoint URLs for `'mainnet'`, `'holesky'`, and `'sepolia'`.

    *Example Format (Use your actual URLs):*
    ```python
    BLOCKCHAIN_RPC = {
        'ethereum': {
            'mainnet': 'https://YOUR_MAINNET_ENDPOINT_URL?key=YOUR_API_KEY',
            # Optional testnets:
            'holesky': 'https://YOUR_HOLESKY_ENDPOINT_URL?key=YOUR_API_KEY',
            'sepolia': 'https://YOUR_SEPOLIA_ENDPOINT_URL?key=YOUR_API_KEY'
        }
    }
    ```

### 🌐 Step 2: Enable Required GCP APIs

Ensure the following APIs are **enabled** in your GCP Project *before* running this cell for authentication and Google Sheets integration to work correctly:

*   **Blockchain Node Engine API:** ([Enable Link](Coming Soon))
*   **Google Drive API:** Required for Google Sheets export/import functionality via `gspread`. ([Enable Link](https://console.cloud.google.com/apis/library/drive.googleapis.com))
*   **Google Sheets API:** Required for Google Sheets export/import functionality via `gspread`. ([Enable Link](https://console.cloud.google.com/apis/library/sheets.googleapis.com))

> #### **💡Tip: Follow README for Video Instructions.**

### 🔐 Step 3: Run the Cell & Authenticate

When you execute the code cell below:

1.  It will define constants (like PYUSD contract addresses) and configurations (trace settings).
2.  It will attempt to **authenticate** your Google account (via a pop-up in Colab) to grant access to the enabled GCP services needed for Google Sheets (Drive, Sheets). Follow the prompts.
3.  It will initialize `web3.py` clients using your provided RPC endpoints.
4.  It will initialize a client for Google Sheets (`gspread`).
5.  It will perform **connection tests** (check RPC node block height) and display a status summary.

> **⚠️ IMPORTANT:** Double-check that you have replaced the placeholder `GCP_PROJECT_ID` and the **full** `BLOCKCHAIN_RPC` URLs before running. The notebook relies heavily on a successful connection to the **Ethereum Mainnet** endpoint via GCP for most subsequent analysis.

In [ ]:
# =============================================================================================
# 📋 Configuration and Authentication for Blockchain Analytics
# =============================================================================================
# This cell configures & Authenticates for blockchain data analysis.

# Import necessary libraries
import google.auth

try:
    from google.colab import auth
except ImportError:
    auth = None
    print("Note: Not running in Google Colab, standard gcloud auth will be used if available.")

import gspread
from web3 import Web3
from web3.middleware import geth_poa_middleware
from rich.console import Console
from rich.theme import Theme
from rich.progress import Progress, SpinnerColumn, TextColumn, TimeElapsedColumn
import time
import os

# Auto-adapting color theme that works well in both light and dark terminals
custom_theme = Theme({
    "info": "cyan3",
    "success": "spring_green3",
    "warning": "gold3",
    "error": "red3",
    "highlight": "royal_blue1"
})

# Ensure console is created with the theme
console = Console(theme=custom_theme)

# =============================================================================================
# Contract Configuration: PYUSD Stablecoin Addresses
# =============================================================================================

# Main PYUSD Contract addresses (used for querying transactions and events)
PYUSD_PROXY = Web3.to_checksum_address('0x6c3ea9036406852006290770bedfcaba0e23a0e8')
PYUSD_IMPLEMENTATION = Web3.to_checksum_address('0x8EcaE0B0402E29694B3Af35d5943D4631Ee568dC')
SUPPLY_CONTROL_PROXY = Web3.to_checksum_address('0x31d9bDEa6F104606C954f8FE6ba614F1BD347Ec3')
SUPPLY_CONTROL_IMPLEMENTATION = Web3.to_checksum_address('0xFaB5891ED867a1195303251912013b92c4fc3a1D')

# PYUSD Contract Registry with implementation contracts
PYUSD_CONTRACTS = {
    PYUSD_PROXY.lower(): "PYUSD Token",
    PYUSD_IMPLEMENTATION.lower(): "PYUSD Implementation",
    SUPPLY_CONTROL_PROXY.lower(): "Supply Control",
    SUPPLY_CONTROL_IMPLEMENTATION.lower(): "Supply Control Impl"
}

# Define event topics first
TRANSFER_EVENT_TOPIC = Web3.keccak(text="Transfer(address,address,uint256)").hex()
APPROVAL_EVENT_TOPIC = Web3.keccak(text="Approval(address,address,uint256)").hex()
PAUSED_EVENT_TOPIC = Web3.keccak(text="Paused(address)").hex()
UNPAUSED_EVENT_TOPIC = Web3.keccak(text="Unpaused(address)").hex()

# Comprehensive PYUSD configuration
PYUSD_CONFIG = {
    'ethereum': {
        'address': PYUSD_PROXY,
        'implementation': PYUSD_IMPLEMENTATION,
        'decimals': 6,
        'symbol': 'PYUSD',
        'deployment_block': 15921958,
        'transfer_event_topic': TRANSFER_EVENT_TOPIC,
        'approval_event_topic': APPROVAL_EVENT_TOPIC,
        'pause_event_topic': PAUSED_EVENT_TOPIC,
        'unpause_event_topic': UNPAUSED_EVENT_TOPIC
    }
}

PYUSD_ADDRESS_LOWER_ETH = PYUSD_CONFIG['ethereum']['address'].lower()

# PYUSD Function Signature Registry
PYUSD_SIGNATURES = {
    '0xa9059cbb': {"name": "transfer(address,uint256)", "type": "function", "category": "token_movement"},
    '0x095ea7b3': {"name": "approve(address,uint256)", "type": "function", "category": "allowance"},
    '0x23b872dd': {"name": "transferFrom(address,address,uint256)", "type": "function", "category": "token_movement"},
    '0x40c10f19': {"name": "mint(address,uint256)", "type": "function", "category": "supply_change"},
    '0x42966c68': {"name": "burn(uint256)", "type": "function", "category": "supply_change"},
    '0x18160ddd': {"name": "totalSupply()", "type": "function", "category": "view"},
    '0x70a08231': {"name": "balanceOf(address)", "type": "function", "category": "view"},
    '0xdd62ed3e': {"name": "allowance(address,address)", "type": "function", "category": "view"},
    '0x313ce567': {"name": "decimals()", "type": "function", "category": "view"},
    '0x06fdde03': {"name": "name()", "type": "function", "category": "view"},
    '0x95d89b41': {"name": "symbol()", "type": "function", "category": "view"},
    '0x8456cb59': {"name": "pause()", "type": "function", "category": "control"},
    '0x3f4ba83a': {"name": "unpause()", "type": "function", "category": "control"},
    '0x5c975abb': {"name": "paused()", "type": "function", "category": "view"},
    '0xf2fde38b': {"name": "transferOwnership(address)", "type": "function", "category": "admin"},
    '0x8da5cb5b': {"name": "owner()", "type": "function", "category": "view"},
    '0x715018a6': {"name": "renounceOwnership()", "type": "function", "category": "admin"}
}

# PYUSD Event Signature Registry with decoders - using the defined event topics
PYUSD_EVENTS = {
    TRANSFER_EVENT_TOPIC: {
        "name": "Transfer(address,address,uint256)",
        "decoder": lambda topics, data: {
            "from": Web3.to_checksum_address('0x' + topics[1][-40:]),
            "to": Web3.to_checksum_address('0x' + topics[2][-40:]),
            "value": int(data, 16)
        }
    },
    APPROVAL_EVENT_TOPIC: {
        "name": "Approval(address,address,uint256)",
        "decoder": lambda topics, data: {
            "owner": Web3.to_checksum_address('0x' + topics[1][-40:]),
            "spender": Web3.to_checksum_address('0x' + topics[2][-40:]),
            "value": int(data, 16)
        }
    },
    PAUSED_EVENT_TOPIC: {
        "name": "Paused(address)",
        "decoder": lambda topics, data: {
            "account": Web3.to_checksum_address('0x' + topics[1][-40:]) if len(topics) > 1 else None
        }
    },
    UNPAUSED_EVENT_TOPIC: {
        "name": "Unpaused(address)",
        "decoder": lambda topics, data: {
            "account": Web3.to_checksum_address('0x' + topics[1][-40:]) if len(topics) > 1 else None
        }
    }
}

# Tracing configurations
TRACE_CONFIGS = {
    "callTracer": {
        "withLog": True,
        "enableReturnData": True,
        "enableMemory": True,
        "enableStack": True
    },
    "structLog": {
        "disableStorage": False,
        "disableMemory": False,
        "disableStack": False,
        "fullStorage": True
    }
}

# Gas analysis categories
GAS_CATEGORIES = {
    "token_movement": ["transfer", "transferFrom"],
    "supply_change": ["mint", "burn"],
    "allowance": ["approve", "increaseAllowance", "decreaseAllowance"],
    "control": ["pause", "unpause"],
    "admin": ["transferOwnership", "renounceOwnership", "addMinter", "removeMinter"],
    "view": ["balanceOf", "allowance", "totalSupply", "decimals", "name", "symbol", "paused", "owner"],
    "other": []
}


# =============================================================================================
# Data Source Configuration: Google Cloud & Blockchain RPC
# =============================================================================================

# Replace "YOUR_PROJECT_ID" with your actual GCP Project ID (needed for GSheets auth scope)
GCP_PROJECT_ID = "YOUR_PROJECT_ID"
if "YOUR_PROJECT_ID" in GCP_PROJECT_ID or not GCP_PROJECT_ID:
    console.print("[error]🚨 CRITICAL: Please replace placeholder or provide your actual GCP Project ID in GCP_PROJECT_ID.", style="bold red")

# Replace "YOUR_MAINNET_BLOCKCHAIN_RPC_URL", "YOUR_HOLESKY_BLOCKCHAIN_RPC_URL", "YOUR_SEPOLIA_BLOCKCHAIN_RPC_URL" with your FULL RPC endpoint URLs including your API key.
# e.g., 'mainnet': 'https://blockchain.googleapis.com/v1/projects/...........'
BLOCKCHAIN_RPC = {
    'ethereum': {
        'holesky': 'YOUR_HOLESKY_BLOCKCHAIN_RPC_URL',
        'mainnet': 'YOUR_MAINNET_BLOCKCHAIN_RPC_URL',
        'sepolia': 'YOUR_SEPOLIA_BLOCKCHAIN_RPC_URL'
    }
}

# Basic Validation checks for placeholders in URLs
rpc_urls_valid = True
if 'ethereum' not in BLOCKCHAIN_RPC or not BLOCKCHAIN_RPC['ethereum']: # Check if ethereum key exists and has entries
    console.print("[error]🚨 CRITICAL: BLOCKCHAIN_RPC['ethereum'] is missing or empty.", style="bold red")
    rpc_urls_valid = False
else:
    for network, url in BLOCKCHAIN_RPC.get('ethereum', {}).items():
        # More robust check for placeholders or incomplete URLs
        if not url or "xxx" in url.lower() or "/v1/" not in url or "key=" not in url or url.endswith("key="):
            console.print(f"[error]🚨 CRITICAL: RPC URL for '{network}' seems invalid or uses placeholder ('{url}'). Use full URL with API key.", style="bold red")
            rpc_urls_valid = False

# Transaction tracing configuration (for detailed transaction analysis)
# Increased timeout for complex traces
DEFAULT_TRACE_CONFIG = {
    'tracer': 'callTracer',
    'timeout': '120s',
    'tracerConfig': {
        'onlyTopCall': False,
        'withLog': True,
    }
}

STRUCTLOG_TRACE_CONFIG = {
    'tracer': 'structLog',
    # 'timeout': '120s',
}

# =============================================================================================
# Global Client Variables
# =============================================================================================
gc_sheets = None
w3_clients = {}

# =============================================================================================
# Utility Functions: Testing & System Verification
# =============================================================================================

def authenticate_gcp(progress, task_id):
    """Authenticate to GCP, initialize Google Sheets client (updates progress descriptively)."""
    global gc_sheets
    progress.update(task_id, description="[info]Initiating GCP Authentication...")
    if not GCP_PROJECT_ID or "project_id" in GCP_PROJECT_ID:
         progress.update(task_id, description="[error]GCP Auth Failed (No Project ID)")
         return False
    auth_success = False
    gs_init_success = False
    effective_project_id = GCP_PROJECT_ID

    try:
        # Step 1: Authentication (Colab vs. Default)
        creds = None
        if auth: # If google.colab.auth was imported successfully
            progress.update(task_id, description="[info]Waiting for Colab user authentication...")
            auth.authenticate_user(project_id=effective_project_id) # Use project ID here if needed by Colab auth context
            auth_success = True
            progress.update(task_id, description="[info]Colab user authenticated. Getting credentials...")
            creds, _ = google.auth.default() # Get default credentials after Colab auth
        else:
            # Attempt standard ADC (Application Default Credentials) - works in VM, Cloud Shell, local gcloud auth login
            progress.update(task_id, description="[info]Attempting default GCP authentication...")
            try:
                creds, inferred_project_id = google.auth.default()
                if not creds:
                     raise Exception("Could not get default credentials.")
                auth_success = True
                progress.update(task_id, description="[info]Default GCP credentials obtained.")
            except Exception as adc_error:
                progress.update(task_id, description="[error]Default GCP Authentication Failed!")
                console.print(f"❌ Default GCP Auth error: {adc_error}", style="error")
                return False # Hard stop if auth fails

        # Ensure we have credentials before proceeding
        if not creds:
             progress.update(task_id, description="[error]Credentials not obtained after auth attempt.")
             return False

        # Step 2: Initialize Google Sheets Client
        progress.update(task_id, description="[info]Initializing Google Sheets client...")
        gc_sheets = gspread.authorize(creds)
        # Perform a minimal check (e.g., list spreadsheets) if needed, but authorize usually suffices
        # gc_sheets.list_spreadsheet_files(max_results=1)
        gs_init_success = True

        progress.update(task_id, description="[success]GCP Authentication & GSheets Client Initialized")
        return True # Overall success

    except Exception as e:
        error_stage = "GCP Setup Error!"
        if not auth_success:
            error_stage = "GCP Authentication Failed!"
        elif not gs_init_success:
            error_stage = "Google Sheets Client Init Failed!"
        progress.update(task_id, description=f"[error]{error_stage}")
        console.print(f"❌ {error_stage}: {str(e)}", style="error")
        if not gs_init_success: gc_sheets = None
        return False


def initialize_all_web3_clients(progress, task_id):
    """Initializes Web3 clients for all networks (updates progress descriptively)."""
    global w3_clients
    progress.update(task_id, description="[info]Initializing Web3 Clients...")
    w3_clients = {}
    success_count = 0
    total_networks = 0
    if 'ethereum' not in BLOCKCHAIN_RPC or not BLOCKCHAIN_RPC['ethereum']:
        progress.update(task_id, description="[error]No valid RPC Config found!")
        return False
    total_networks = len(BLOCKCHAIN_RPC['ethereum'])
    network_statuses = []

    for network, rpc_url in BLOCKCHAIN_RPC['ethereum'].items():
        progress.update(task_id, description=f"[info]Connecting to {network.capitalize()}...")
        time.sleep(0.1) # Small delay for visual update
        # Find the original url value before the loop modified it
        original_url = BLOCKCHAIN_RPC.get('ethereum', {}).get(network, "")
        is_invalid_url = not original_url or "xxx" in original_url.lower() or "/v1/" not in original_url or "key=" not in original_url or original_url.endswith("key=")
        if is_invalid_url:
            w3_clients[network] = None
            network_statuses.append(f"{network.capitalize()}:[error]Skipped (Invalid URL)[/error]")
            progress.update(task_id, description=f"[warning]Skipping {network.capitalize()} (Invalid URL)...")
            continue
        try:
            # Increased timeout slightly for potentially slower network conditions
            provider = Web3.HTTPProvider(rpc_url, request_kwargs={'timeout': 120})
            w3_client = Web3(provider)
            # Inject middleware only if needed (e.g., for PoA testnets like Goerli, Rinkeby - less relevant for Mainnet/Sepolia/Holesky now)
            # Check chain ID if necessary to decide on middleware, but generally safe to add
            w3_client.middleware_onion.inject(geth_poa_middleware, layer=0)

            # Test connection with get_block_number
            block_num = w3_client.eth.get_block_number()
            w3_clients[network] = w3_client
            success_count += 1
            network_statuses.append(f"{network.capitalize()}:[success]OK (Block: {block_num:,})[/success]")
            progress.update(task_id, description=f"[info]Connecting... ({success_count}/{total_networks} OK)")
        except Exception as e:
            error_short = type(e).__name__
            # Add more detail for common errors
            if "Max retries exceeded" in str(e): error_short = "ConnectionTimeout"
            elif "Failed to establish a new connection" in str(e): error_short = "ConnectionRefused"
            network_statuses.append(f"{network.capitalize()}:[error]{error_short}[/error]")
            w3_clients[network] = None
            progress.update(task_id, description=f"[warning]Failed {network.capitalize()} ({error_short})...")
            console.print(f"[warning]Web3 connection error for {network.capitalize()}: {e}", style="warning")


    final_web3_status = f"[success]Web3 Clients Initialized ({success_count}/{total_networks} OK)"
    if success_count == 0:
        final_web3_status = "[error]Web3 Client Init Failed (All Networks)"
    elif success_count < total_networks:
         final_web3_status = f"[warning]Web3 Clients Initialized ({success_count}/{total_networks} OK)"
    progress.update(task_id, description=final_web3_status)

    # Return True if mainnet client initialized successfully
    return w3_clients.get('mainnet') is not None

# =============================================================================================
# Main Execution: System Initialization and Status Check (with Progress)
# =============================================================================================

# --- Title Display ---
console.print("\n✨ Configuration and Authentication ✨", style="bold cyan3")
console.print("───────────────────────────────────────", style="cyan3")

# Initialize status variables
gcp_auth_success = False
any_web3_success = False
mainnet_ready = False

# Use Rich Progress for initialization steps
# Set transient=True to make the progress bar disappear on completion
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    TimeElapsedColumn(),
    console=console, # Ensure progress uses the themed console
    transient=True # Make spinner disappear when done
) as progress:
    # Add tasks for each step (total=1 means they complete in one update)
    auth_task = progress.add_task("GCP Authentication...", total=1)
    web3_task = progress.add_task("Initializing Web3 Clients...", total=1)

    # --- Run Initialization Steps (functions update progress description) ---
    gcp_auth_success = authenticate_gcp(progress, auth_task)
    progress.update(auth_task, completed=1) # Mark as done

    if rpc_urls_valid:
        any_web3_success = initialize_all_web3_clients(progress, web3_task)
    else:
        progress.update(web3_task, description="[error]Skipped Web3 Init (Invalid URLs)")
    progress.update(web3_task, completed=1)


# --- Get Final Status ---
mainnet_ready = any_web3_success # mainnet_ready directly reflects if mainnet client succeeded
overall_success = gcp_auth_success and mainnet_ready

# --- Display Intermediate Success Messages (Now that progress is done) ---
if gcp_auth_success:
    console.print(f"✓ User authentication successful!", style="success")
    if gc_sheets: console.print(f"✓ Google Sheets client initialized", style="success")
else:
    console.print(f"❌ GCP authentication failed.", style="error")

# Updated Web3 success message based on w3_clients dictionary
if w3_clients:
    connected_nets = [net.capitalize() for net, client in w3_clients.items() if client]
    if connected_nets:
        console.print(f"✓ Web3 clients connected: {', '.join(connected_nets)}", style="success")
    elif rpc_urls_valid: # Only show warning if init was attempted but failed all
         console.print("[warning]Web3 clients initialized, but none connected successfully.", style="warning")
else:
     # This case occurs if rpc_urls_valid was False
     console.print("[error]Web3 client initialization skipped due to invalid RPC URLs.", style="error")


# --- Display Final System Status Summary ---
console.print("\n\n📊 System Status Summary", style="highlight")
# RPC Status
if w3_clients:
    for network, client in w3_clients.items():
        connected = client is not None
        status_msg = "[red]Failed/Skipped[/red]" # Default if not connected
        if connected:
             try:
                 block_num = client.eth.block_number # Get block number again for final status
                 block_num_str = f"(Block #{block_num:,})"
                 status_msg = f"[green]Connected[/green] {block_num_str}"
             except Exception as e:
                  # If client exists but fails here, mark as unresponsive
                  status_msg = f"[orange3]Unresponsive ({type(e).__name__})[/orange3]"
                  w3_clients[network] = None # Clear bad client for consistency
                  console.print(f"[warning]RPC ({network.capitalize()}) became unresponsive: {e}", style="warning")
        else:
            # Add reason if skipped due to invalid URL during init
            original_url = BLOCKCHAIN_RPC.get('ethereum', {}).get(network, "")
            if not original_url or "xxx" in original_url.lower():
                 status_msg = "[red]Skipped (Invalid URL)[/red]"

        console.print(f"  • Ethereum RPC ({network.capitalize()}): {status_msg}")
elif not rpc_urls_valid:
    console.print("  • Ethereum RPC: [red]Skipped (Invalid URLs)[/red]")
else:
    console.print("  • Ethereum RPC: [red]Initialization Failed[/red]")

# GCP Status
status_auth = "[green]Successful[/green]" if gcp_auth_success else "[red]Failed[/red]"
console.print(f"  • GCP Authentication: {status_auth}")

# Google Sheets Status
status_gs = "[green]Initialized[/green]" if gc_sheets else "[red]Not Initialized[/red]"
if not gcp_auth_success: # Also mark skipped if auth failed
    status_gs = "[yellow]Skipped[/yellow]"
console.print(f"  • Google Sheets Client: {status_gs}")

# --- Final Ready/Failure Message with Checkmark ---
if overall_success:
    console.print("\n\n[bold green]✓ Configuration Complete:[/bold green] System Ready for Ethereum Blockchain Analytics (via RPC) and Google Sheets")
    # Optional warnings for testnets can still be useful
    if 'holesky' in w3_clients and w3_clients.get('holesky') is None: console.print("  [warning](Note: Holesky testnet connection failed/skipped/unresponsive)", style="warning")
    if 'sepolia' in w3_clients and w3_clients.get('sepolia') is None: console.print("  [warning](Note: Sepolia testnet connection failed/skipped/unresponsive)", style="warning")
else:
     failure_reasons = []
     if not gcp_auth_success: failure_reasons.append("GCP Auth Failed")
     if not w3_clients.get('mainnet'): failure_reasons.append("Mainnet RPC Connection Failed/Skipped")
     if gcp_auth_success and not gc_sheets: failure_reasons.append("Google Sheets Client Failed") # Check if GSheets failed despite auth success
     if not rpc_urls_valid: failure_reasons.append("Invalid RPC URL Placeholders")


     reason_str = ', '.join(failure_reasons) if failure_reasons else "Unknown Issues"
     console.print(f"\n\n[bold red]❌ Configuration Failed:[/bold red] System setup encountered issues ({reason_str}). Review status messages above.")

## 1.1 🎯 Analysis Targets & Utility Functions
---

This cell handles target validation and prepares the essential utilities needed for transaction/block analysis:

1.  **🌐 Network Verification:**
    *   Verifies connectivity to Ethereum Mainnet and available testnets
    *   Displays current block heights and chain IDs
    *   Attempts reconnection if mainnet client is unavailable

2.  **🎯 Define Analysis Targets:**
    *   Set the specific Ethereum Mainnet **transaction hash** (`TARGET_TX_HASH`) or **block identifier** (`TARGET_BLOCK_IDENTIFIER`) you wish to analyze
    *   **ACTION:** Modify these values to analyze different transactions or blocks relevant to PYUSD
    *   Targets are automatically validated for proper format and existence on-chain

3.  **🔧 Helper Function Library:**
    *   Initializes essential functions used throughout the notebook for:
        *   Making raw RPC requests (`make_rpc_request`) to Ethereum nodes
        *   Decoding PYUSD-specific function calls and events
        *   Formatting blockchain values (ETH/PYUSD amounts, gas)
        *   Handling addresses and transaction data
        *   Creating visualizations for transaction analysis

> **Note:** If a target validation fails, specific diagnostic information will be displayed to help troubleshoot the issue.

In [ ]:
# =============================================================================================
# 🎯 Target Selection & Helper Functions for Tracing
# =============================================================================================
# This cell validates targets, initializes tracing configurations, and sets up helper functions.

# Ensure web3 clients dictionary is loaded from the previous cell
if 'w3_clients' not in locals() or not isinstance(w3_clients, dict):
     raise NameError("Web3 clients dictionary 'w3_clients' not initialized. Please run '🔑 Configuration & Authentication: Connecting to GCP and Ethereum RPC' Cell")

# --- Network Status Verification ---
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    console=console,
    transient=True
) as progress:
    verification_task = progress.add_task("[info]Verifying network connections...", total=1)

    w3_mainnet = w3_clients.get('mainnet')
    if not w3_mainnet or not w3_mainnet.is_connected():
         # Attempt re-initialization with visual feedback
         progress.update(verification_task, description="[warning]⚠️ Mainnet client not found. Attempting re-initialization...")
         web3_task = progress.add_task("[info]Reinitializing Web3 clients...", total=1)
         initialize_all_web3_clients(progress, web3_task)
         progress.update(web3_task, completed=1)

         # Check if reconnection succeeded
         w3_mainnet = w3_clients.get('mainnet')
         if not w3_mainnet or not w3_mainnet.is_connected():
              raise ConnectionError("Cannot proceed without a connected Mainnet Web3 client. Check your RPC endpoints.")

    # Get testnet client if available (for testing function implementations)
    progress.update(verification_task, description="[info]Checking for testnet availability...")
    w3_testnet = w3_clients.get('sepolia') or w3_clients.get('holesky') # Prefer Sepolia if available
    if w3_testnet and w3_testnet.is_connected():
        # Find the network name ('holesky' or 'sepolia')
        testnet_name = next((name for name in ['sepolia', 'holesky'] if w3_clients.get(name) == w3_testnet), None)
    else:
        testnet_name = None # No connections with testnet

    progress.update(verification_task, completed=1)

# --- Network Status Table ---
console.print("[bold cyan3 size=20]🌐 Network Connections[/]", justify="left")
console.print("────────────────────────", style="cyan3")

network_table = Table(show_header=True, header_style="bold cyan3")
network_table.add_column("Network", style="dim")
network_table.add_column("Status", justify="center")
network_table.add_column("Block Height", justify="right")
network_table.add_column("Chain ID", justify="right")

# Add Mainnet info
latest_block_mainnet = w3_mainnet.eth.block_number if w3_mainnet else "N/A"
chain_id_mainnet = w3_mainnet.eth.chain_id if w3_mainnet else "N/A"
network_table.add_row(
    "Ethereum Mainnet",
    "[spring_green3]Connected[/spring_green3]" if w3_mainnet and w3_mainnet.is_connected() else "[red3]Disconnected[/red3]",
    f"{latest_block_mainnet:,}" if isinstance(latest_block_mainnet, int) else str(latest_block_mainnet),
    str(chain_id_mainnet)
)

# Add Testnet info if available
if testnet_name:
    latest_block_testnet = w3_testnet.eth.block_number if w3_testnet else "N/A"
    chain_id_testnet = w3_testnet.eth.chain_id if w3_testnet else "N/A"
    network_table.add_row(
        f"{testnet_name.capitalize()} Testnet",
        "[spring_green3]Connected[/spring_green3]" if w3_testnet and w3_testnet.is_connected() else "[red3]Disconnected[/red3]",
        f"{latest_block_testnet:,}" if isinstance(latest_block_testnet, int) else str(latest_block_testnet),
        str(chain_id_testnet)
    )
else:
    network_table.add_row("Testnet", "[gold3]Not Available[/gold3]", "N/A", "N/A")

# Display the network connection table
console.print(network_table)
console.print("\n\n")  # Empty lines for better spacing

############################################################
# 🎯 DEFINE YOUR ETHEREUM ANALYSIS TARGET
# Set ONE of these variables (comment out the other):
############################################################

# Option 1: Transaction Hash to analyze
TARGET_TX_HASH = "YOUR_TARGET_TX_HASH"

# Option 2: Block Number/Hash to analyze
TARGET_BLOCK_IDENTIFIER = YOUR_TARGET_BLOCK_NUMBER

# Note: If both are set, TARGET_TX_HASH takes priority
############################################################

# Import necessary packages for helper functions
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from graphviz import Digraph
from IPython.display import display, Javascript
from hexbytes import HexBytes
from rich.panel import Panel
from rich.syntax import Syntax
from rich.table import Table
import json
from datetime import datetime

# Plotly configuration for Colab
try:
    display(Javascript('''
        require.config({
            paths: {
                plotly: 'https://cdn.plot.ly/plotly-latest.min.js'
            }
        });
    '''))
except Exception as e:
    console.print(f"[warning]Could not re-configure Plotly: {e}", style="warning")

# --- Target Validation Functions ---
def validate_tx_hash(tx_hash):
    """Validates a transaction hash with detailed diagnostics"""
    if not tx_hash:
        return {
            'valid': False,
            'status': "Missing",
            'status_color': "red3",
            'details': "Transaction hash not provided"
        }

    if not isinstance(tx_hash, str):
        return {
            'valid': False,
            'status': "Invalid Type",
            'status_color': "red3",
            'details': f"Expected string, got {type(tx_hash).__name__}"
        }

    if not tx_hash.startswith('0x'):
        return {
            'valid': False,
            'status': "Invalid Format",
            'status_color': "red3",
            'details': "Transaction hash must start with '0x'"
        }

    if len(tx_hash) < 66:
        return {
            'valid': False,
            'status': "Too Short",
            'status_color': "red3",
            'details': f"Length is {len(tx_hash)}, should be 66 characters"
        }

    if len(tx_hash) > 66:
        return {
            'valid': False,
            'status': "Too Long",
            'status_color': "red3",
            'details': f"Length is {len(tx_hash)}, should be 66 characters (including '0x')"
        }

    # Check if characters are valid hex
    try:
        int(tx_hash[2:], 16)
    except ValueError:
        return {
            'valid': False,
            'status': "Invalid Hex",
            'status_color': "red3",
            'details': "Contains non-hexadecimal characters"
        }

    # At this point, the format is valid, try to retrieve it
    try:
        tx_receipt = w3_mainnet.eth.get_transaction_receipt(tx_hash[:66])  # Truncate to valid length for retrieval
        tx_details = w3_mainnet.eth.get_transaction(tx_hash[:66])

        # Format gas info
        gas_used = tx_receipt.get('gasUsed', 0)
        gas_limit = tx_details.get('gas', 0)
        gas_percentage = (gas_used / gas_limit * 100) if gas_limit else 0

        # Check if transaction was successful
        is_success = tx_receipt.get('status') == 1

        return {
            'valid': True,
            'status': "Found" if is_success else "Failed Transaction",
            'status_color': "spring_green3" if is_success else "gold3",
            'details': f"Block: {tx_receipt.get('blockNumber')}, Gas: {gas_used:,}/{gas_limit:,} ({gas_percentage:.1f}%)",
            'data': {
                'receipt': tx_receipt,
                'transaction': tx_details,
                'success': is_success
            }
        }
    except Exception as e:
        return {
            'valid': True,  # Format is valid but transaction not found
            'status': "Valid Format, Not Found",
            'status_color': "gold3",
            'details': f"Error: {str(e)}"  # No truncation
        }

def validate_block_identifier(block_id):
    """Validates a block identifier with detailed diagnostics"""
    if block_id is None:
        return {
            'valid': False,
            'status': "Missing",
            'status_color': "red3",
            'details': "Block identifier not provided"
        }

    # For integer block numbers
    if isinstance(block_id, int):
        if block_id < 0:
            return {
                'valid': False,
                'status': "Negative Number",
                'status_color': "red3",
                'details': "Block number cannot be negative"
            }

        # Check if block is within realistic range
        latest_block = w3_mainnet.eth.block_number if w3_mainnet else None
        if latest_block and block_id > latest_block:
            return {
                'valid': False,
                'status': "Future Block",
                'status_color': "red3",
                'details': f"Block {block_id:,} exceeds current mainnet height ({latest_block:,})"
            }

        # Block is valid by format, try to retrieve it
        try:
            block_details = w3_mainnet.eth.get_block(block_id)

            # Format timestamp from Unix timestamp
            block_time = datetime.fromtimestamp(block_details.timestamp)

            return {
                'valid': True,
                'status': "Found",
                'status_color': "spring_green3",
                'details': f"Time: {block_time.strftime('%Y-%m-%d %H:%M:%S')}, TX Count: {len(block_details['transactions'])}",
                'data': block_details
            }
        except Exception as e:
            return {
                'valid': True,  # Format is valid but block not found
                'status': "Valid Format, Not Found",
                'status_color': "gold3",
                'details': f"Error: {str(e)}"  # No truncation
            }

    # For hex string or hash
    if isinstance(block_id, str):
        if not block_id.startswith('0x'):
            return {
                'valid': False,
                'status': "Invalid Format",
                'status_color': "red3",
                'details': "Block hash must start with '0x'"
            }

        # Check if characters are valid hex
        try:
            int(block_id[2:], 16)
        except ValueError:
            return {
                'valid': False,
                'status': "Invalid Hex",
                'status_color': "red3",
                'details': "Contains non-hexadecimal characters"
            }

        # Block hash is valid by format, try to retrieve it
        try:
            block_details = w3_mainnet.eth.get_block(block_id)

            # Format timestamp from Unix timestamp
            block_time = datetime.fromtimestamp(block_details.timestamp)

            return {
                'valid': True,
                'status': "Found",
                'status_color': "spring_green3",
                'details': f"Time: {block_time.strftime('%Y-%m-%d %H:%M:%S')}, TX Count: {len(block_details['transactions'])}",
                'data': block_details
            }
        except Exception as e:
            return {
                'valid': True,  # Format is valid but block not found
                'status': "Valid Format, Not Found",
                'status_color': "gold3",
                'details': f"Error: {str(e)}"  # No truncation
            }

    # If not int or str
    return {
        'valid': False,
        'status': "Invalid Type",
        'status_color': "red3",
        'details': f"Expected int or string, got {type(block_id).__name__}"
    }

# --- Target Verification & Information Retrieval with progress indicator ---
with Progress(
    SpinnerColumn(),
    TextColumn("[progress.description]{task.description}"),
    console=console,
    transient=True
) as progress:
    validate_task = progress.add_task("[info]Validating analysis targets...", total=2)

    # Validate transaction hash
    progress.update(validate_task, description="[info]Analyzing transaction hash...")
    if 'TARGET_TX_HASH' in locals() and TARGET_TX_HASH:
        tx_validation = validate_tx_hash(TARGET_TX_HASH)
        tx_valid = tx_validation['valid']
    else:
        tx_valid = False
        tx_validation = {
            'status': "Skipped",
            'status_color': "gold3",
            'details': "No transaction hash provided"
        }

    progress.update(validate_task, advance=1)

    # Validate block identifier
    progress.update(validate_task, description="[info]Analyzing block identifier...")
    if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
        block_validation = validate_block_identifier(TARGET_BLOCK_IDENTIFIER)
        block_valid = block_validation['valid']
    else:
        block_valid = False
        block_validation = {
            'status': "Skipped",
            'status_color': "gold3",
            'details': "No block identifier provided"
        }

    progress.update(validate_task, completed=1)

# --- Target Status Display (Key-Value Format) ---
console.print("[bold royal_blue1 size=20]🎯 Analysis Targets[/]", justify="left")
console.print("────────────────────", style="royal_blue1")
console.print("\n")  # Extra spacing

# Transaction Target
console.print("[bold]Target Type - Transaction Hash[/bold]")
if 'TARGET_TX_HASH' in locals() and TARGET_TX_HASH:
    console.print(f"[dim]Value:[/dim] [cyan3]{TARGET_TX_HASH}[/cyan3]")
else:
    console.print("[dim]Value:[/dim] Nil")

console.print(f"[dim]Status:[/dim] [{tx_validation['status_color']}]{tx_validation.get('status', 'Unknown')}[/{tx_validation['status_color']}]")
console.print(f"[dim]Diagnostics:[/dim] {tx_validation.get('details', 'No information available')}")

# Separator
console.print("\n" + "─" * 50 + "\n")

# Block Target
console.print("[bold]Target Type - Block[/bold]")
if 'TARGET_BLOCK_IDENTIFIER' in locals() and TARGET_BLOCK_IDENTIFIER is not None:
    console.print(f"[dim]Value:[/dim] [cyan3]{TARGET_BLOCK_IDENTIFIER}[/cyan3]")
else:
    console.print("[dim]Value:[/dim] Nil")

console.print(f"[dim]Status:[/dim] [{block_validation['status_color']}]{block_validation.get('status', 'Unknown')}[/{block_validation['status_color']}]")
console.print(f"[dim]Diagnostics:[/dim] {block_validation.get('details', 'No information available')}")

console.print("\n")  # Extra spacing

# Show target selection status message
if not tx_valid and not block_valid:
    console.print("[error]⚠️ No valid targets found. Please set either TARGET_TX_HASH or TARGET_BLOCK_IDENTIFIER.[/error]", style="bold red3")
elif tx_valid and block_valid:
    # Check if both targets exist but the transaction validation found the transaction and the block validation found the block
    if tx_validation.get('status') == "Found" and block_validation.get('status') == "Found":
        console.print("[info]ℹ️ Both transaction and block targets are found. Transaction analysis will take priority.[/info]", style="bold royal_blue1")
    else:
        console.print("[info]ℹ️ Both transaction and block targets are provided but at least one wasn't found. Valid target will be used.[/info]", style="bold royal_blue1")
elif tx_valid:
    if tx_validation.get('status') == "Found":
        console.print("[info]✓ Transaction target is valid and found. Ready for analysis.[/info]", style="bold spring_green3")
    else:
        console.print("[warning]⚠️ Transaction target format is valid but transaction wasn't found. Check the hash.[/warning]", style="bold gold3")
elif block_valid:
    if block_validation.get('status') == "Found":
        console.print("[info]✓ Block target is valid and found. Ready for analysis.[/info]", style="bold spring_green3")
    else:
        console.print("[warning]⚠️ Block target format is valid but block wasn't found. Check the number/hash.[/warning]", style="bold gold3")

console.print("\n")  # Empty line for better spacing

# --- Helper Functions ---
def make_rpc_request(method, params, network='mainnet'):
    """Helper function to make raw RPC requests via the specified network's provider."""
    w3_client = w3_clients.get(network)
    if not w3_client or not w3_client.is_connected():
        console.print(f"[error]Web3 client for '{network}' not available or not connected.", style="error")
        return None
    try:
        response = w3_client.provider.make_request(method, params)
        if 'error' in response:
            console.print(f"[error]RPC Error ({method} on {network}): {response['error']['message']} (Code: {response['error']['code']})", style="error")
            return None
        return response.get('result')
    except Exception as e:
        console.print(f"[error]Exception during RPC call ({method} on {network}): {str(e)}", style="error")
        return None

# --- PYUSD Contract Helpers ---
def is_pyusd_contract(address):
    """Checks if an address is a known PYUSD contract."""
    if not address:
        return False
    address_lower = address.lower() if isinstance(address, str) else str(address).lower()
    return address_lower in PYUSD_CONTRACTS

def get_contract_name(address):
    """Gets the friendly name for a contract address."""
    if not address:
        return "Unknown"
    address_lower = address.lower() if isinstance(address, str) else str(address).lower()
    return PYUSD_CONTRACTS.get(address_lower, "Other Contract")

def decode_pyusd_function(input_data):
    """Decodes PYUSD function calls using the signature registry."""
    if not input_data or input_data == '0x':
        return "Empty call data"

    method_sig = input_data[:10]
    if method_sig in PYUSD_SIGNATURES:
        return PYUSD_SIGNATURES[method_sig]["name"]
    return f"Unknown function: {method_sig}"

def get_function_category(input_data):
    """Gets the category of a function from its input data."""
    if not input_data or input_data == '0x':
        return "other"

    method_sig = input_data[:10]
    if method_sig in PYUSD_SIGNATURES:
        return PYUSD_SIGNATURES[method_sig]["category"]
    return "other"

def decode_pyusd_event(topic0, topics, data):
    """Decodes PYUSD events using the event registry."""
    if not topic0 or topic0 not in PYUSD_EVENTS:
        return {"name": "Unknown event", "details": "Cannot decode"}

    event_info = PYUSD_EVENTS[topic0]

    try:
        decoded = event_info["decoder"](topics, data)
        return {
            "name": event_info["name"],
            "decoded": decoded
        }
    except Exception as e:
        return {
            "name": event_info["name"],
            "error": str(e)
        }

# --- Formatting Helpers ---
def format_value_pyusd(value_raw):
    """Formats raw PYUSD value (int) to decimal string."""
    if value_raw is None: return "0 PYUSD"
    try:
        decimals = PYUSD_CONFIG['ethereum']['decimals']
        value_float = int(value_raw) / (10**decimals)
        return f"{value_float:,.{decimals}f} PYUSD"
    except (ValueError, TypeError):
        return "Invalid PYUSD Value"

def format_value_eth(value_wei):
    """Formats Wei value to Ether string."""
    if value_wei is None: return "0 ETH"
    try:
        # Ensure value_wei is int or can be converted (handle hex strings from traces)
        if isinstance(value_wei, str):
            value_int = int(value_wei, 16)
        else:
             value_int = int(value_wei)
        # Use mainnet client for formatting ETH value regardless of target network
        if w3_mainnet: # Check if mainnet client exists
            return f"{w3_mainnet.from_wei(value_int, 'ether'):.6f} ETH"
        else:
            return f"{value_int / 1e18:.6f} ETH (approx)" # Fallback if mainnet client missing
    except (ValueError, TypeError, AttributeError):
        return "Invalid ETH Value"

def format_gas(gas):
    """Formats gas value (int or hex string)."""
    if gas is None: return "N/A"
    try:
       gas_int = int(gas, 16) if isinstance(gas, str) and gas.startswith('0x') else int(gas)
       return f"{gas_int:,}"
    except (ValueError, TypeError):
        return "Invalid Gas Value"

def shorten_address(address):
    """Shortens an Ethereum address for display."""
    if not isinstance(address, str) or not address.startswith('0x') or len(address) != 42:
        return str(address) # Return original if not a valid address string
    return f"{address[:6]}...{address[-4:]}"

def display_json(data, title="JSON Output"):
    """Pretty prints JSON data using Rich."""
    if data is None:
        console.print(f"[warning]{title}: No data to display.", style="warning")
        return
    try:
        # Use default=str to handle potential non-serializable types like HexBytes
        json_str = json.dumps(data, indent=2, default=str)
        console.print(Panel(Syntax(json_str, "json", theme="default", line_numbers=False),
                      title=title, border_style="cyan3", expand=False))
    except Exception as e:
        console.print(f"[error]Could not display JSON for {title}: {e}", style="error")
        # Fallback to printing raw data (limited length)
        try:
            raw_str = str(data)
            console.print(raw_str[:1000] + ("..." if len(raw_str) > 1000 else ""))
        except Exception:
            console.print("[error]Could not print raw data fallback.", style="error")

# --- Visualization Helpers ---
def create_gas_usage_chart(gas_data, title="Gas Usage Distribution"):
    """Creates a pie chart showing gas usage distribution."""
    try:
        fig = px.pie(gas_data, values='gas_used', names='category',
                      title=title)
        fig.update_layout(template="plotly_white")
        return fig
    except Exception as e:
        console.print(f"[warning]Could not create gas usage chart: {e}", style="warning")
        return None

def create_call_sequence_chart(calls_df, highlight_pyusd=True):
    """Creates a bar chart showing the sequence of calls with gas usage."""
    try:
        if highlight_pyusd:
            fig = px.bar(calls_df, x='id', y='gasUsed', color='is_pyusd',
                         title='Gas Usage by Call Sequence',
                         hover_data=['type', 'depth', 'from', 'to'])
        else:
            fig = px.bar(calls_df, x='id', y='gasUsed',
                         title='Gas Usage by Call Sequence',
                         hover_data=['type', 'depth', 'from', 'to'])

        fig.update_layout(template="plotly_white")
        return fig
    except Exception as e:
        console.print(f"[warning]Could not create call sequence chart: {e}", style="warning")
        return None

# --- Final Tracing Configuration Setup ---
# Set up trace configuration based on targets
if tx_valid and tx_validation.get('status') == "Found":
    # If valid transaction hash found, prepare for transaction tracing
    ACTIVE_TRACE_CONFIG = DEFAULT_TRACE_CONFIG.copy()
    console.print(f"[info]Trace configuration ready for transaction analysis", style="info")
elif block_valid and block_validation.get('status') == "Found":
    # If valid block found, prepare for block tracing
    ACTIVE_TRACE_CONFIG = STRUCTLOG_TRACE_CONFIG.copy()
    console.print(f"[info]Trace configuration ready for block analysis", style="info")
else:
    # If no valid targets, set a default trace config
    ACTIVE_TRACE_CONFIG = DEFAULT_TRACE_CONFIG.copy()
    console.print(f"[warning]Using default trace configuration (no valid target found)", style="warning")

# Final status message
status_icon = "✓" if (tx_valid and tx_validation.get('status') == "Found") or (block_valid and block_validation.get('status') == "Found") else "⚠️"
status_style = "bold spring_green3" if status_icon == "✓" else "bold gold3"
console.print(f"[{status_style}]{status_icon} Target selection and helper functions initialized[/{status_style}]")